<a href="https://colab.research.google.com/github/amitgupta005/logistic_project/blob/main/Income_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
# Importing library for logistic regression
from sklearn.linear_model import LogisticRegression

# Importing performance metrics - accuracy score & confusion matrix
from sklearn.metrics import accuracy_score,confusion_matrix

In [ ]:
inc=pd.read_csv(r"/content/drive/MyDrive/Colab Notebooks/csv files/final/income(1).csv")
inc.head()

,age,JobType,EdType,maritalstatus,occupation,relationship,race,gender,capitalgain,capitalloss,hoursperweek,nativecountry,SalStat
0,45,Private,HS-grad,Divorced,Adm-clerical,Not-in-family,White,Female,0,0,28,United-States,"less than or equal to 50,000"
1,24,Federal-gov,HS-grad,Never-married,Armed-Forces,Own-child,White,Male,0,0,40,United-States,"less than or equal to 50,000"
2,44,Private,Some-college,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,40,United-States,"greater than 50,000"
3,27,Private,9th,Never-married,Craft-repair,Other-relative,White,Male,0,0,40,Mexico,"less than or equal to 50,000"
4,20,Private,Some-college,Never-married,Sales,Not-in-family,White,Male,0,0,35,United-States,"less than or equal to 50,000"


In [ ]:
inc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31978 entries, 0 to 31977
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   age            31978 non-null  int64 
 1   JobType        31978 non-null  object
 2   EdType         31978 non-null  object
 3   maritalstatus  31978 non-null  object
 4   occupation     31978 non-null  object
 5   relationship   31978 non-null  object
 6   race           31978 non-null  object
 7   gender         31978 non-null  object
 8   capitalgain    31978 non-null  int64 
 9   capitalloss    31978 non-null  int64 
 10  hoursperweek   31978 non-null  int64 
 11  nativecountry  31978 non-null  object
 12  SalStat        31978 non-null  object
dtypes: int64(4), object(9)
memory usage: 3.2+ MB


In [ ]:
unique_values = {}
for column in inc.columns:
    unique_values[column] = ' '.join(map(str, inc[column].unique()))

# Print unique values for each column
for column, values in unique_values.items():
    print(f' $$ Unique values in {column}: {values}')

 $$ Unique values in age: 45 24 44 27 20 51 17 19 60 40 30 38 23 32 54 50 37 52 49 22 35 67 46 33 43 28 41 25 63 59 34 26 72 62 58 47 39 53 57 21 42 74 36 29 31 69 18 65 55 48 71 70 66 56 61 68 64 73 81 78 76 90 88 75 77 80 82 79 83 84 85 86 87
 $$ Unique values in JobType:  Private  Federal-gov  ?  Self-emp-inc  Self-emp-not-inc  Local-gov  State-gov  Without-pay  Never-worked
 $$ Unique values in EdType:  HS-grad  Some-college  9th  11th  Assoc-voc  Assoc-acdm  Bachelors  1st-4th  Masters  10th  12th  5th-6th  Prof-school  Doctorate  7th-8th  Preschool
 $$ Unique values in maritalstatus:  Divorced  Never-married  Married-civ-spouse  Widowed  Married-spouse-absent  Separated  Married-AF-spouse
 $$ Unique values in occupation:  Adm-clerical  Armed-Forces  Prof-specialty  Craft-repair  Sales  Exec-managerial  Machine-op-inspct  ?  Transport-moving  Farming-fishing  Tech-support  Other-service  Handlers-cleaners  Protective-serv  Priv-house-serv
 $$ Unique values in relationship:  Not-in

In [ ]:
inc["race"].value_counts()

 White                 27430
 Black                  3028
 Asian-Pac-Islander      956
 Amer-Indian-Eskimo      311
 Other                   253
Name: race, dtype: int64

In [ ]:
# Categorising age to different ranges
# Define the bin edges and labels for age groups
bin_edges = [17, 30, 40, 50, 60, 90]  # Define your age group boundaries
bin_labels = ['17-29', '30-39', '40-49', '50-59', '60-90']  # Labels for the groups

# Create a new column 'age_group' with the age groups
inc['age_group'] = pd.cut(inc['age'], bins=bin_edges, labels=bin_labels, right=False)

# Now, 'age_group' will contain the age group for each row in the DataFrame.

In [ ]:
# Replace "?" with NaN in the 'JobType' column
inc['JobType'] = inc['JobType'].replace(" ?", np.nan)

In [ ]:
# Replace "?" with NaN in the 'occupation' column
inc['occupation'] = inc['occupation'].replace(" ?", np.nan)

In [ ]:
inc.isna().sum()

age                 0
JobType          1809
EdType              0
maritalstatus       0
occupation       1816
relationship        0
race                0
gender              0
capitalgain         0
capitalloss         0
hoursperweek        0
nativecountry       0
SalStat             0
age_group          42
dtype: int64

In [ ]:
# Replace null values in 'occupation' with "Unemployed" when 'JobType' is "Never-worked"
inc['occupation'] = inc.apply(lambda row: " Unemployed" if row['JobType'] == " Never-worked" and pd.isna(row['occupation']) else row['occupation'],axis=1)

In [ ]:
inc = inc.drop("age", axis=1)

In [ ]:
inc.isna().sum()

JobType          1809
EdType              0
maritalstatus       0
occupation       1809
relationship        0
race                0
gender              0
capitalgain         0
capitalloss         0
hoursperweek        0
nativecountry       0
SalStat             0
age_group          42
dtype: int64

In [ ]:
# The dataset is conatining a large number of rows so we can afforf to remove rows containing null values
inc=inc.dropna()

In [ ]:
inc.drop_duplicates(keep='first',inplace=True)
inc.shape

<ipython-input-19-15e003fd5394>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inc.drop_duplicates(keep='first',inplace=True)


(21314, 13)

In [ ]:
import ydata_profiling as pp
pp.ProfileReport(inc)

ModuleNotFoundError: No module named 'ydata_profiling'

In [ ]:
inc['SalStat'] = inc['SalStat'].replace({' greater than 50,000': 1, ' less than or equal to 50,000': 0})

In [ ]:
# Define your custom bins/ranges
bins = [1, 11, 21, 31, 41, 51, 61, 71, 81, 91]

# Define labels for the bins
labels = ['1-10', '11-20', '21-30', '31-40', '41-50', '51-60', '61-70', '71-80', '81-90']

# Assuming 'hoursperweek' is the column you want to convert
inc['hoursperweek_range'] = pd.cut(inc['hoursperweek'], bins=bins, labels=labels, right=False)

# Now, 'hoursperweek_range' contains the ranges you specified

In [ ]:
# Replace values other than " White" with " Non-White" in the 'race' column
inc['new_race_column'] = np.where(inc['race'] == ' White', 'White', 'Non-White')

In [ ]:
# Assuming 'age_group' and 'hoursperweek_range' are the columns you want to convert
inc['age_group'] = inc['age_group'].astype('object')
inc['hoursperweek_range'] = inc['hoursperweek_range'].astype('object')

In [ ]:
inc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21314 entries, 0 to 31976
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   JobType             21314 non-null  object
 1   EdType              21314 non-null  object
 2   maritalstatus       21314 non-null  object
 3   occupation          21314 non-null  object
 4   relationship        21314 non-null  object
 5   race                21314 non-null  object
 6   gender              21314 non-null  object
 7   capitalgain         21314 non-null  int64 
 8   capitalloss         21314 non-null  int64 
 9   hoursperweek        21314 non-null  int64 
 10  nativecountry       21314 non-null  object
 11  SalStat             21314 non-null  int64 
 12  age_group           21314 non-null  object
 13  hoursperweek_range  21213 non-null  object
 14  new_race_column     21314 non-null  object
dtypes: int64(4), object(11)
memory usage: 2.6+ MB


In [ ]:
inc = inc.drop("hoursperweek", axis=1)

In [ ]:
inc = inc.drop("race", axis=1)

In [ ]:
inc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21314 entries, 0 to 31976
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   JobType             21314 non-null  object
 1   EdType              21314 non-null  object
 2   maritalstatus       21314 non-null  object
 3   occupation          21314 non-null  object
 4   relationship        21314 non-null  object
 5   gender              21314 non-null  object
 6   capitalgain         21314 non-null  int64 
 7   capitalloss         21314 non-null  int64 
 8   nativecountry       21314 non-null  object
 9   SalStat             21314 non-null  int64 
 10  age_group           21314 non-null  object
 11  hoursperweek_range  21213 non-null  object
 12  new_race_column     21314 non-null  object
dtypes: int64(3), object(10)
memory usage: 2.3+ MB


In [ ]:
inc.sample(5)

,JobType,EdType,maritalstatus,occupation,relationship,gender,capitalgain,capitalloss,nativecountry,SalStat,age_group,hoursperweek_range,new_race_column
18875,Private,HS-grad,Married-civ-spouse,Sales,Husband,Male,0,0,United-States,0,30-39,41-50,White
3434,Private,HS-grad,Never-married,Adm-clerical,Unmarried,Female,0,0,United-States,0,40-49,31-40,White
10460,Private,Masters,Divorced,Exec-managerial,Unmarried,Male,0,0,United-States,1,50-59,31-40,White
13803,Private,HS-grad,Never-married,Exec-managerial,Unmarried,Male,0,0,United-States,0,17-29,31-40,White
8813,Private,Assoc-acdm,Divorced,Sales,Not-in-family,Male,0,0,United-States,0,30-39,51-60,White


Model Fitting 1

In [ ]:
inc2 = inc.copy()

In [ ]:
new_data2=pd.get_dummies(inc2, drop_first=True)

In [ ]:
new_data2.shape

(21314, 104)

In [ ]:
# x2 = new_data2.drop(columns = ["SalStat"], axis = 1)

In [ ]:
# Storing the column names
columns_list2=list(new_data2.columns)
print(columns_list2)

['capitalgain', 'capitalloss', 'SalStat', 'JobType_ Local-gov', 'JobType_ Never-worked', 'JobType_ Private', 'JobType_ Self-emp-inc', 'JobType_ Self-emp-not-inc', 'JobType_ State-gov', 'JobType_ Without-pay', 'EdType_ 11th', 'EdType_ 12th', 'EdType_ 1st-4th', 'EdType_ 5th-6th', 'EdType_ 7th-8th', 'EdType_ 9th', 'EdType_ Assoc-acdm', 'EdType_ Assoc-voc', 'EdType_ Bachelors', 'EdType_ Doctorate', 'EdType_ HS-grad', 'EdType_ Masters', 'EdType_ Preschool', 'EdType_ Prof-school', 'EdType_ Some-college', 'maritalstatus_ Married-AF-spouse', 'maritalstatus_ Married-civ-spouse', 'maritalstatus_ Married-spouse-absent', 'maritalstatus_ Never-married', 'maritalstatus_ Separated', 'maritalstatus_ Widowed', 'occupation_ Armed-Forces', 'occupation_ Craft-repair', 'occupation_ Exec-managerial', 'occupation_ Farming-fishing', 'occupation_ Handlers-cleaners', 'occupation_ Machine-op-inspct', 'occupation_ Other-service', 'occupation_ Priv-house-serv', 'occupation_ Prof-specialty', 'occupation_ Protective

In [ ]:
# Separating the input names from data
features2=list(set(columns_list2)-set(['SalStat']))
print(features2)

['age_group_40-49', 'EdType_ Prof-school', 'nativecountry_ China', 'nativecountry_ Ecuador', 'age_group_50-59', 'nativecountry_ Columbia', 'nativecountry_ Taiwan', 'nativecountry_ Germany', 'occupation_ Prof-specialty', 'EdType_ 5th-6th', 'nativecountry_ Hong', 'nativecountry_ Cuba', 'nativecountry_ Laos', 'EdType_ Assoc-acdm', 'nativecountry_ Haiti', 'nativecountry_ Thailand', 'nativecountry_ Mexico', 'EdType_ 1st-4th', 'occupation_ Protective-serv', 'JobType_ Without-pay', 'nativecountry_ Hungary', 'occupation_ Transport-moving', 'nativecountry_ Honduras', 'EdType_ 7th-8th', 'EdType_ Some-college', 'capitalloss', 'nativecountry_ India', 'nativecountry_ El-Salvador', 'occupation_ Other-service', 'EdType_ 9th', 'gender_ Male', 'nativecountry_ Puerto-Rico', 'occupation_ Machine-op-inspct', 'hoursperweek_range_81-90', 'maritalstatus_ Widowed', 'maritalstatus_ Married-spouse-absent', 'occupation_ Priv-house-serv', 'nativecountry_ United-States', 'occupation_ Sales', 'EdType_ Preschool', '

In [ ]:
# Storing the output values in y
y2=new_data2['SalStat'].values

# Storing the values from input features
x2 = new_data2[features2].values   # values gives pandas array

In [ ]:
# Splitting the data into train and test
train_x2,test_x2,train_y2,test_y2 = train_test_split(x2,y2,test_size=0.3, random_state=0)

In [ ]:
# Make an instance of the Model
logistic = LogisticRegression(solver='lbfgs', max_iter=27000)

# Fitting the values for x and y
logistic.fit(train_x2,train_y2)
logistic.coef_
logistic.intercept_

array([-2.56509486])

In [ ]:
# Prediction from test data
prediction2 = logistic.predict(test_x2)
print(prediction2)


[1 0 0 ... 0 0 1]


In [ ]:
# Confusion matrix
confusion_matrix3 = confusion_matrix(test_y2, prediction2)
print(confusion_matrix3)

[[4436  363]
 [ 636  960]]


In [ ]:
# Calculating the accuracy
accuracy_score2=accuracy_score(test_y2, prediction2)
print(accuracy_score2)

0.8437842064112588


Now, some more work to do

In [ ]:
# Drop the 'nativecountry' column
inc = inc.drop(columns=['nativecountry'])

KeyError: "['nativecountry'] not found in axis"

In [ ]:
con_data = [feature for feature in inc.columns if inc[feature].dtype == "object" or inc[feature].dtype == "category"]
con_data

['JobType',
 'EdType',
 'maritalstatus',
 'occupation',
 'relationship',
 'gender',
 'age_group',
 'hoursperweek_range',
 'new_race_column']

In [ ]:
from scipy.stats import chi2_contingency

# Create an empty DataFrame to store the results
independence_results = pd.DataFrame(columns=['Feature', 'Chi-squared', 'p-value'])

# Iterate through each categorical column
for column in con_data:
    # Create a contingency table
    contingency_table = pd.crosstab(inc[column], inc['SalStat'])

    # Perform the chi-squared test
    chi2, p, _, _ = chi2_contingency(contingency_table)

    # Store the results in the DataFrame
    independence_results = independence_results.append({'Feature': column, 'Chi-squared': chi2, 'p-value': p}, ignore_index=True)

# Sort the results by p-value (smaller p-value means more significant)
independence_results.sort_values(by='p-value', ascending=True, inplace=True)

# Print or analyze the results
print(independence_results)


              Feature  Chi-squared        p-value
1              EdType  2672.545196   0.000000e+00
2       maritalstatus  4100.073214   0.000000e+00
3          occupation  2274.527433   0.000000e+00
4        relationship  4287.434397   0.000000e+00
6           age_group  1463.134943   0.000000e+00
7  hoursperweek_range  1391.951904  3.111640e-295
5              gender   915.216631  4.829904e-201
0             JobType   553.499091  2.493258e-115
8     new_race_column   203.069188   4.467775e-46


<ipython-input-51-fe2295936b72>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  independence_results = independence_results.append({'Feature': column, 'Chi-squared': chi2, 'p-value': p}, ignore_index=True)
<ipython-input-51-fe2295936b72>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  independence_results = independence_results.append({'Feature': column, 'Chi-squared': chi2, 'p-value': p}, ignore_index=True)
<ipython-input-51-fe2295936b72>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  independence_results = independence_results.append({'Feature': column, 'Chi-squared': chi2, 'p-value': p}, ignore_index=True)
<ipython-input-51-fe2295936b72>:15: FutureWarning: The frame.append method is deprecated and will be removed 

In [ ]:
inc = inc.drop(['capitalgain', 'capitalloss'], axis=1)
inc = inc.drop("gender", axis=1) #As it had high correlation with the relationship column and the relationship column had more unique values so I think that it may be a better fit

In [ ]:
# Capital gain and capital loss the columns are removed because they had very high number of zero values for example the total number of 2 1000 and the zero values are about 18000

In [ ]:
inc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21314 entries, 0 to 31976
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   JobType             21314 non-null  object
 1   EdType              21314 non-null  object
 2   maritalstatus       21314 non-null  object
 3   occupation          21314 non-null  object
 4   relationship        21314 non-null  object
 5   SalStat             21314 non-null  int64 
 6   age_group           21314 non-null  object
 7   hoursperweek_range  21213 non-null  object
 8   new_race_column     21314 non-null  object
dtypes: int64(1), object(8)
memory usage: 1.6+ MB


Model Fitting 2

In [ ]:
new_data=pd.get_dummies(inc, drop_first=True)

In [ ]:
# Storing the column names
columns_list=list(new_data.columns)
print(columns_list)

['SalStat', 'JobType_ Local-gov', 'JobType_ Never-worked', 'JobType_ Private', 'JobType_ Self-emp-inc', 'JobType_ Self-emp-not-inc', 'JobType_ State-gov', 'JobType_ Without-pay', 'EdType_ 11th', 'EdType_ 12th', 'EdType_ 1st-4th', 'EdType_ 5th-6th', 'EdType_ 7th-8th', 'EdType_ 9th', 'EdType_ Assoc-acdm', 'EdType_ Assoc-voc', 'EdType_ Bachelors', 'EdType_ Doctorate', 'EdType_ HS-grad', 'EdType_ Masters', 'EdType_ Preschool', 'EdType_ Prof-school', 'EdType_ Some-college', 'maritalstatus_ Married-AF-spouse', 'maritalstatus_ Married-civ-spouse', 'maritalstatus_ Married-spouse-absent', 'maritalstatus_ Never-married', 'maritalstatus_ Separated', 'maritalstatus_ Widowed', 'occupation_ Armed-Forces', 'occupation_ Craft-repair', 'occupation_ Exec-managerial', 'occupation_ Farming-fishing', 'occupation_ Handlers-cleaners', 'occupation_ Machine-op-inspct', 'occupation_ Other-service', 'occupation_ Priv-house-serv', 'occupation_ Prof-specialty', 'occupation_ Protective-serv', 'occupation_ Sales', '

In [ ]:
# Separating the input names from data
features=list(set(columns_list)-set(['SalStat']))
print(features)

['hoursperweek_range_41-50', 'age_group_40-49', 'EdType_ Prof-school', 'occupation_ Exec-managerial', 'maritalstatus_ Separated', 'relationship_ Wife', 'age_group_50-59', 'EdType_ Doctorate', 'occupation_ Prof-specialty', 'EdType_ 5th-6th', 'relationship_ Other-relative', 'hoursperweek_range_71-80', 'JobType_ Never-worked', 'EdType_ Assoc-acdm', 'EdType_ Assoc-voc', 'occupation_ Tech-support', 'EdType_ 1st-4th', 'occupation_ Protective-serv', 'JobType_ Without-pay', 'relationship_ Own-child', 'occupation_ Transport-moving', 'JobType_ Self-emp-not-inc', 'EdType_ 7th-8th', 'EdType_ Some-college', 'occupation_ Unemployed', 'occupation_ Other-service', 'EdType_ 9th', 'occupation_ Machine-op-inspct', 'hoursperweek_range_61-70', 'JobType_ Self-emp-inc', 'hoursperweek_range_81-90', 'maritalstatus_ Widowed', 'maritalstatus_ Married-spouse-absent', 'occupation_ Armed-Forces', 'relationship_ Unmarried', 'occupation_ Farming-fishing', 'occupation_ Priv-house-serv', 'occupation_ Sales', 'age_group

In [ ]:
# Storing the output values in y
y=new_data['SalStat'].values

# Storing the values from input features
x = new_data[features].values   # values gives pandas array

In [ ]:
# Splitting the data into train and test
train_x,test_x,train_y,test_y = train_test_split(x,y,test_size=0.3, random_state=0)

In [ ]:
# Make an instance of the Model
logistic2 = LogisticRegression(solver='lbfgs', max_iter=27000)

# Fitting the values for x and y
logistic2.fit(train_x,train_y)
logistic2.coef_
logistic2.intercept_


array([-4.80565842])

In [ ]:
# Prediction from test data
prediction = logistic2.predict(test_x)
print(prediction)


[1 0 1 ... 1 0 0]


In [ ]:
# Calculating the accuracy
accuracy_score=accuracy_score(test_y, prediction)
print(accuracy_score)

0.8242376856919469


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

roc_auc = roc_auc_score(test_y, logistic2.predict_proba(test_x)[:,1])

print("Accuracy : ", roc_auc)
print("Classification Report :\n ",classification_report(test_y,prediction))
print("confusion Matrix:\n", confusion_matrix(test_y, prediction))

Accuracy :  0.8784140623490378
Classification Report :
                precision    recall  f1-score   support

           0       0.87      0.91      0.89      4799
           1       0.67      0.57      0.62      1596

    accuracy                           0.82      6395
   macro avg       0.77      0.74      0.75      6395
weighted avg       0.82      0.82      0.82      6395

confusion Matrix:
 [[4354  445]
 [ 679  917]]


# Decision Tree Classifier

In [ ]:

from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.tree import DecisionTreeClassifier, export_text

In [ ]:
# Create a Decision Tree Classifier
decision_tree = DecisionTreeClassifier(max_depth = 10,random_state=42)

# Train the decision tree on the training data
decision_tree.fit(train_x, train_y)

# Make predictions on the testing data
y_pred = decision_tree.predict(test_x)

# Evaluate the accuracy of the model
accuracy = sum(y_pred == test_y) / len(test_y)
print("Accuracy:", accuracy)

Accuracy: 0.8054730258014073


In [ ]:
# Define the hyperparameter grid
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 5, 10, 15],
    'min_samples_split': [50, 100, 200, 500],
    'max_features': [None, 'sqrt', 'log2']
}

# Create a GridSearchCV object
grid_search = GridSearchCV(estimator=decision_tree, param_grid=param_grid, cv=5, scoring='accuracy')

# Fit the model to the training data
grid_search.fit(train_x, train_y)

# Print the best hyperparameters
print("Best Hyperparameters:", grid_search.best_params_)

# Evaluate the model on the test set
test_accuracy = grid_search.best_estimator_.score(test_x, test_y)
print("Test Accuracy:", test_accuracy)

Best Hyperparameters: {'criterion': 'gini', 'max_depth': None, 'max_features': None, 'min_samples_split': 500}
Test Accuracy: 0.8070367474589523


# Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [ ]:
# Create a Random Forest Classifier
rf_classifier = RandomForestClassifier(random_state=42)

# Train the Random Forest on the training data
rf_classifier.fit(train_x, train_y)

# Make predictions on the test data
predictions = rf_classifier.predict(test_x)

# Evaluate the performance
accuracy = accuracy_score(test_y, predictions)
print("Accuracy:", accuracy)

# Print classification report
print("Classification Report:")
print(classification_report(test_y, predictions))


Accuracy: 0.7882720875684128
Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.87      0.86      4799
           1       0.58      0.54      0.56      1596

    accuracy                           0.79      6395
   macro avg       0.72      0.70      0.71      6395
weighted avg       0.78      0.79      0.79      6395



In [ ]:
# Hyperparameter Tuning with GridSearchCV (Optional)
param_grid = {
    'n_estimators': [50, 100, 200, 500],
    'max_depth': [None, 10,15, 20],
    'min_samples_split': [50, 100, 200, 500],
    'max_features': [None, 'sqrt', 'log2']
}

In [ ]:
grid_search = GridSearchCV(estimator=RandomForestClassifier(random_state=42),
                           param_grid=param_grid, cv=5, scoring='accuracy')

grid_search.fit(train_x, train_y)


KeyboardInterrupt: 

In [ ]:
# Print the best hyperparameters
print("Best Hyperparameters (GridSearchCV):", grid_search.best_params_)

# Make predictions with the best model from GridSearchCV
best_rf_model = grid_search.best_estimator_
best_predictions = best_rf_model.predict(test_x)

# Evaluate the performance of the best model
best_accuracy = accuracy_score(test_y, best_predictions)
print("Accuracy with Best Model:", best_accuracy)

In [ ]:
# Hyperparameter Tuning with RandomizedSearchCV (Optional)
random_search = RandomizedSearchCV(estimator=RandomForestClassifier(random_state=42),
                                   param_distributions=param_grid, n_iter=10, cv=5, scoring='accuracy', random_state=42)

random_search.fit(train_x, train_y)

# Print the best hyperparameters
print("Best Hyperparameters (RandomizedSearchCV):", random_search.best_params_)

# Make predictions with the best model from RandomizedSearchCV
best_rf_model_random = random_search.best_estimator_
best_predictions_random = best_rf_model_random.predict(test_x)

# Evaluate the performance of the best model from RandomizedSearchCV
best_accuracy_random = accuracy_score(test_y, best_predictions_random)
print("Accuracy with Best Model (RandomizedSearchCV):", best_accuracy_random)


Best Hyperparameters (RandomizedSearchCV): {'n_estimators': 100, 'min_samples_split': 50, 'max_features': 'log2', 'max_depth': 20}
Accuracy with Best Model (RandomizedSearchCV): 0.8172009382329946


In [ ]:
# Create a Random Forest Classifier after hyperparameter tuning
rf_classifier = RandomForestClassifier(random_state=42, n_estimators = 100, min_samples_split = 50, max_features = 'log2', max_depth = 20)

# Train the Random Forest on the training data
rf_classifier.fit(train_x, train_y)

# Make predictions on the test data
predictions = rf_classifier.predict(test_x)

# Evaluate the performance
accuracy = accuracy_score(test_y, predictions)
print("Accuracy:", accuracy)

Accuracy: 0.8172009382329946
